# Installing Requirements Packages

#### one-time set up of Bioconda with the following commands.

In [ ]:
!conda config --add channels defaults
!conda config --add channels bioconda
!conda config --add channels conda-forge
!conda config --set channel_priority strict

#### Install the following packages in conda environments


In [ ]:
# Install mamba in the base conda environment
# Note: This step might be optional, as mamba is not always necessary
!conda install -n base --override-channels -c conda-forge mamba 'python_abi=*=*cp*'
 

In [ ]:
# Install the ete3 package using conda
!conda install ete3 --quiet --yes

!conda install -c conda-forge biopython

# Install the centrifuge package from the bioconda channel using conda
!conda install bioconda::centrifuge


# Install r-remotes and bioconductor-rsamtools using mamba
!mamba install --yes --quiet r-remotes bioconductor-rsamtools

# Install ipywidgets using pip
!pip install ipywidgets

!pip install epi2melabs

# Downloading data 

In [4]:
!wget -O mappings.tar.gz https://osf.io/g5at8/download


--2024-03-05 19:47:37--  https://osf.io/g5at8/download
Resolving osf.io (osf.io)... 35.190.84.173
Connecting to osf.io (osf.io)|35.190.84.173|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://files.au-1.osf.io/v1/resources/gry6e/providers/osfstorage/5e4daa9c1949df0191563b5f?action=download&direct&version=1 [following]
--2024-03-05 19:47:37--  https://files.au-1.osf.io/v1/resources/gry6e/providers/osfstorage/5e4daa9c1949df0191563b5f?action=download&direct&version=1
Resolving files.au-1.osf.io (files.au-1.osf.io)... 35.241.28.215
Connecting to files.au-1.osf.io (files.au-1.osf.io)|35.241.28.215|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 235807592 (225M) [application/octet-stream]
Saving to: ‘mappings.tar.gz’

mappings.tar.gz     100%[===================>] 224.88M  13.9MB/s    in 18s     

2024-03-05 19:47:57 (12.8 MB/s) - ‘mappings.tar.gz’ saved [235807592/235807592]



In [5]:
!tar xvzf mappings.tar.gz

tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
mappings/evol1.sorted.dedup.q20.bam
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
mappings/evol1.sorted.unmapped.R1.fastq.gz
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
mappings/evol1.sorted.unmapped.R2.fastq.gz
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
mappings/evol2.sorted.dedup.q20.bam


In [ ]:
!gunzip mappings/evol1.sorted.unmapped.R1.fastq.gz

# Downloading Centrifuge Index

 h+p+v+c: human genome, prokaryotic genomes, and viral genomes including SARS-CoV-2 genomes.

In [9]:
!wget https://zenodo.org/records/3732127/files/h+p+v+c.tar.gz


--2024-03-05 22:21:39--  https://zenodo.org/records/3732127/files/h+p+v+c.tar.gz
Resolving zenodo.org (zenodo.org)... 188.184.103.159, 188.184.98.238, 188.185.79.172, ...
Connecting to zenodo.org (zenodo.org)|188.184.103.159|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20737377933 (19G) [application/octet-stream]
Saving to: ‘h+p+v+c.tar.gz’

h+p+v+c.tar.gz      100%[===================>]  19.31G  13.1MB/s    in 26m 6s  

2024-03-05 22:47:47 (12.6 MB/s) - ‘h+p+v+c.tar.gz’ saved [20737377933/20737377933]



In [7]:
!tar -xvzf h+p+v+c.tar.gz

hpvc.1.cf
hpvc.2.cf
hpvc.3.cf
hpvc.4.cf


# Run Centrifuge

In [17]:
!centrifuge -x hpvc -U mappings/evol1.sorted.unmapped.R1.fastq --report-file report1.txt -S results1.txt

In [7]:
from Bio import Entrez
import pandas as pd
import os
from tqdm import tqdm
from IPython.display import display

# Set your email address for the NCBI server
Entrez.email = "your_email@example.com"

def fetch_taxa(taxid_list):

    try:
        # fetch taxa
        handle = Entrez.efetch('taxonomy', id=taxid_list, rettype='xml')
        response = Entrez.read(handle)

        # filter response
        classified_taxonomy = {}
        for index, entry in enumerate(response):
            # no lineage in keys
            if 'LineageEx' not in entry.keys():
                classified_taxonomy[index] = "No_Taxa"
                continue
            # this can be modified
            taxa_dict = {}
            for taxa_class in entry["LineageEx"]:
                taxa_dict[taxa_class["Rank"]] = taxa_class["ScientificName"]
            classified_taxonomy[entry['TaxId']] = taxa_dict

        return classified_taxonomy
    except Exception as e:
        print(f"Error fetching taxonomic info: {e}")
        return None


d = pd.read_csv(os.path.join("report.txt"), sep='\t')

lineage = []

# Get unique TaxIDs from the DataFrame
unique_taxIDs = d['taxID'].unique()

for taxid in tqdm(unique_taxIDs):
    taxonomic_info = fetch_taxa([taxid])

    if taxonomic_info:
        lineage.append(taxonomic_info)

# Create a DataFrame to map unique TaxIDs to Genus names

# Flatten the list of dictionaries
flattened_list = [{'tax_id': key, **value} for entry in lineage for key, value in entry.items()]

# Create a DataFrame from the flattened list
mapping_df = pd.DataFrame(flattened_list)
mapping_df['taxID'] = mapping_df['tax_id'].astype('int64') 

# Merge the mapping DataFrame with the original DataFrame
result_df = pd.merge(d, mapping_df, on='taxID', how='left')

# Group by the mapped genus column
genus_id = result_df.groupby('genus') \
    .agg(numReads=('numReads', 'sum')) \
    .sort_values('numReads', ascending=False) \
    .reset_index()

read_filter = 1
common_genera = sum(genus_id['numReads'] > read_filter)

print("{} genera identified with >{} reads.".format(common_genera, read_filter))
display(genus_id.head(8))
# printit  # This line might be a mistake; check if you need it



  3%|██████▏                                                                                                                                                                                                             | 4/137 [00:02<01:31,  1.46it/s]

Error fetching taxonomic info: HTTP Error 400: Bad Request


  4%|█████████▎                                                                                                                                                                                                          | 6/137 [00:04<02:02,  1.07it/s]

Error fetching taxonomic info: HTTP Error 400: Bad Request


 45%|█████████████████████████████████████████████████████████████████████████████████████████████▉                                                                                                                     | 61/137 [00:30<00:47,  1.61it/s]

...

 93%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏             | 128/137 [03:34<00:06,  1.35it/s]

Error fetching taxonomic info: HTTP Error 400: Bad Request


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 137/137 [03:39<00:00,  1.60s/it]

Error fetching taxonomic info: HTTP Error 400: Bad Request
21 genera identified with >1 reads.


,genus,numReads
0,Shigella,463
1,Escherichia,458
2,Salmonella,70
3,Homo,44
4,Pseudomonas,11
5,Klebsiella,11
6,Enterobacter,11
7,Staphylococcus,9


# Run Pavian 

In [8]:
!centrifuge-kreport -x hpvc  results.txt > read_classifications.tsv.kraken

Loading taxonomy ...
Loading names file ...
/home/sr3622/miniconda3/bin/centrifuge-inspect:24: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp
Loading nodes file ...
/home/sr3622/miniconda3/bin/centrifuge-inspect:24: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


In [13]:
import ipywidgets as widgets
from epi2melabs.notebook import InputForm, InputSpec

pavian_form = InputForm(
    InputSpec('port', 'Aux. EPI2ME Labs port', widgets.IntText(8889)))
pavian_form.display()

In [ ]:
# running Pavian web server
# !echo "Checking pavian install..."

script = """
ncpus=parallel::detectCores()
options(Ncpus=ncpus)
remotes::install_github("fbreitwieser/pavian", upgrade=T, quiet=T)"""
_script = os.path.expanduser("~/.pavian_install.R")
with open(_script, "w") as fh:
    fh.write(script)
!Rscript $_script
!echo "Done."
!echo "Running pavian..."
port = pavian_form.port
!R -e "pavian::runApp(host='0.0.0.0', port="$port")"

Done.
Running pavian...

R version 4.3.3 (2024-02-29) -- "Angel Food Cake"
Copyright (C) 2024 The R Foundation for Statistical Computing
Platform: x86_64-conda-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> pavian::runApp(host='0.0.0.0', port=8889)
Loading required package: shiny

Listening on http://0.0.0.0:8889
[Mar06 21:20] Started new shiny session #1 (1 session(s) running)
[Mar06 22:12] Exiting session #1
